In [ ]:
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas

     |████████████████████████████████| 16.7 MB 3.0 MB/s 
     |████████████████████████████████| 6.3 MB 8.0 MB/s 
     |████████████████████████████████| 1.0 MB 2.4 MB/s 
     |████████████████████████████████| 1.0 MB 8.8 MB/s 


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 57.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a8ed7378f1d9bf8e5b1076aeb8173d43070a827f489d22fb6f54bae6f1a6e4c6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-07.csv> yellow_tripdata_2019-07.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-08.csv> yellow_tripdata_2019-08.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-09.csv> yellow_tripdata_2019-09.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-10.csv> yellow_tripdata_2019-10.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-11.csv> yellow_tripdata_2019-11.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-12.csv> yellow_tripdata_2019-12.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  557M  100  557M    0     0  94.7M      0  0:00:05  0:00:05 --:--:-- 95.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  536M  100  536M    0     0  94.8M      0  0:00:05  0:00:05 --:--:-- 95.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  580M  100  580M    0     0  95.0M      0  0:00:06  0:00:06 --:--:-- 95.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  638M  100  638M    0     0  94.8M      0  0:00:06  0:00:06 --:--:-- 95.3M
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
file_name=[]
for i in range(7,13):
  if i<10:
    file_name.append('yellow_tripdata_2019-0{}.csv'.format(i))
  else:
    file_name.append('yellow_tripdata_2019-{}.csv'.format(i))
file_name

['yellow_tripdata_2019-07.csv',
 'yellow_tripdata_2019-08.csv',
 'yellow_tripdata_2019-09.csv',
 'yellow_tripdata_2019-10.csv',
 'yellow_tripdata_2019-11.csv',
 'yellow_tripdata_2019-12.csv']

In [ ]:
import geopandas as gpd

In [ ]:
taxi_zone=gpd.read_file('taxizone.geojson')
taxi_zone.head()

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()

In [ ]:
crime='NYPD_Complaint_Data_Historic.csv'
crime_data = sc.textFile(crime, use_unicode=True).cache()

In [ ]:
from datetime import datetime
from datetime import date

In [ ]:
def extract_2019_2020(partID,part):
  if partID==0:
    next(part)

  import csv
  for record in csv.reader(part):
    print(record)
    occ_Date, occ_Time, locID, offense_type, offense_name, law, boro, lat, lon =\
    record[1],record[2],record[5],record[7],record[8],record[12],record[13],record[27],record[28]

    if len(occ_Date)>6:
        d=occ_Date.split('/')
        year=int(d[2])
        year=int(d[2])
        month=int(d[0])
        day=int(d[1])
        g= date(year,month,day)

        if g>date(2019,6,30) and g<date(2020,1,1):
          yield occ_Date,occ_Time,locID,offense_type,offense_name,law,boro,float(lat),float(lon),2019

        if g>date(2020,6,30) and g<date(2021,1,1):
          yield occ_Date,occ_Time,locID,offense_type,offense_name,law,boro,float(lat),float(lon),2020

In [ ]:
test=crime_data.mapPartitionsWithIndex(extract_2019_2020)

In [ ]:
test.count()

446288

In [ ]:
import pandas as pd
cime_class=pd.read_excel('crime_classes.xlsx')
cime_class

,code,class
0,113,Unrelated crimes
1,345,Unrelated crimes
2,238,Unrelated crimes
3,364,Unrelated crimes
4,364,Unrelated crimes
...,...,...
73,353,driving related
74,342,driving related
75,119,driving related
76,881,driving related


In [ ]:
classes=set(list(cime_class['class']))
classes

{'Unable to classlfy',
 'Unrelated crimes',
 'alcohol',
 'dangerous felony',
 'driving related',
 'larceny & fraud',
 'sex related'}

In [ ]:
class_code_list={'Unable to classlfy':[],'Unrelated crimes':[],'alcohol':[], 'dangerous felony':[], 'driving related':[], 'larceny & fraud':[],'sex related':[]}
for row in range(len(cime_class)):
  class_code_list[cime_class.iloc[row][1]].append(str(int(cime_class.iloc[row][0])))


In [ ]:
class_code_list

{'Unable to classlfy': ['126',
  '361',
  '678',
  '365',
  '675',
  '677',
  '677',
  '364',
  '366',
  '676'],
 'Unrelated crimes': ['113',
  '345',
  '238',
  '364',
  '364',
  '350',
  '125',
  '122',
  '571',
  '356',
  '230',
  '120',
  '120',
  '572',
  '349',
  '115',
  '362',
  '234',
  '357'],
 'alcohol': ['346'],
 'dangerous felony': ['118',
  '101',
  '117',
  '106',
  '344',
  '114',
  '235',
  '236',
  '237',
  '125',
  '124',
  '363',
  '103',
  '124',
  '102',
  '124',
  '455'],
 'driving related': ['348', '347', '353', '342', '119', '881'],
 'larceny & fraud': ['107',
  '341',
  '109',
  '110',
  '105',
  '351',
  '112',
  '352',
  '358',
  '340',
  '359',
  '355',
  '121',
  '111',
  '343',
  '232',
  '343',
  '231',
  '345',
  '354'],
 'sex related': ['578', '104', '233', '116', '116']}

In [ ]:
crime_data=test.collect()

In [ ]:
c_data=pd.DataFrame(crime_data,columns=['Date','Time','loc ID','off type','off name','off law type','boro','lat','lon','year'])
c_data

,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year
0,12/31/2019,17:30:00,32,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,40.820927,-73.943324,2019
1,12/29/2019,16:31:00,47,113,FORGERY,FELONY,BRONX,40.885701,-73.861640,2019
2,12/15/2019,18:45:00,109,578,HARRASSMENT 2,VIOLATION,QUEENS,40.742281,-73.819824,2019
3,12/28/2019,01:00:00,47,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,40.875311,-73.847545,2019
4,12/27/2019,22:00:00,9,107,BURGLARY,FELONY,MANHATTAN,40.720759,-73.980466,2019
...,...,...,...,...,...,...,...,...,...,...
446283,09/09/2019,08:30:00,18,578,HARRASSMENT 2,VIOLATION,MANHATTAN,40.767849,-73.987630,2019
446284,12/17/2019,10:00:00,113,109,GRAND LARCENY,FELONY,QUEENS,40.674593,-73.796869,2019
446285,08/21/2019,00:00:00,34,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,40.850460,-73.938470,2019
446286,12/31/2019,02:00:00,60,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,40.573909,-74.000495,2019


In [ ]:
#because this type of offense only account for less than 1%
#and the type itself seems not related to trips
#so we decide to drop them

# but code 125 stands for 2 names, and one of than should be counted
# we only drop rows with off name == OTHER STATE LAWS (NON PENAL LA
# same as type 364

no_use=c_data[c_data['off type'].isin(class_code_list['Unrelated crimes'])]
no_use

,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year
1,12/29/2019,16:31:00,47,113,FORGERY,FELONY,BRONX,40.885701,-73.861640,2019
11,12/22/2019,04:15:00,108,113,FORGERY,FELONY,QUEENS,40.743125,-73.956068,2019
24,12/18/2019,17:00:00,48,113,FORGERY,FELONY,BRONX,40.857523,-73.882319,2019
100,11/19/2019,09:20:00,18,113,FORGERY,FELONY,MANHATTAN,40.757396,-73.982257,2019
344,08/31/2019,00:55:00,109,113,FORGERY,FELONY,QUEENS,40.760803,-73.830744,2019
...,...,...,...,...,...,...,...,...,...,...
446053,12/30/2019,15:01:00,5,125,NYS LAWS-UNCLASSIFIED FELONY,FELONY,MANHATTAN,40.716344,-74.001253,2019
446060,12/28/2019,07:00:00,5,125,NYS LAWS-UNCLASSIFIED FELONY,FELONY,MANHATTAN,40.716344,-74.001253,2019
446068,12/03/2019,15:00:00,5,125,NYS LAWS-UNCLASSIFIED FELONY,FELONY,MANHATTAN,40.716344,-74.001253,2019
446098,12/21/2019,10:05:00,34,125,NYS LAWS-UNCLASSIFIED FELONY,FELONY,MANHATTAN,40.849035,-73.933035,2019


In [ ]:
class_code_list['Unrelated crimes'].remove('125')
class_code_list['Unrelated crimes'].remove('364')
class_code_list['Unrelated crimes'].remove('364')
class_code_list['Unrelated crimes'].remove('345')

In [ ]:
c_data=c_data[c_data['off name']!='OTHER STATE LAWS (NON PENAL LA']
c_data.shape

(446056, 10)

In [ ]:
#class_code_list['Unrelated crimes'].remove('364')
c_data=c_data[~c_data['off type'].isin(class_code_list['Unrelated crimes'])]
c_data

,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year
0,12/31/2019,17:30:00,32,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,40.820927,-73.943324,2019
2,12/15/2019,18:45:00,109,578,HARRASSMENT 2,VIOLATION,QUEENS,40.742281,-73.819824,2019
3,12/28/2019,01:00:00,47,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,40.875311,-73.847545,2019
4,12/27/2019,22:00:00,9,107,BURGLARY,FELONY,MANHATTAN,40.720759,-73.980466,2019
5,12/27/2019,20:10:00,79,117,DANGEROUS DRUGS,FELONY,BROOKLYN,40.695958,-73.937632,2019
...,...,...,...,...,...,...,...,...,...,...
446283,09/09/2019,08:30:00,18,578,HARRASSMENT 2,VIOLATION,MANHATTAN,40.767849,-73.987630,2019
446284,12/17/2019,10:00:00,113,109,GRAND LARCENY,FELONY,QUEENS,40.674593,-73.796869,2019
446285,08/21/2019,00:00:00,34,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,40.850460,-73.938470,2019
446286,12/31/2019,02:00:00,60,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,40.573909,-74.000495,2019


In [ ]:
c_data=c_data[c_data['off name']!='ENDAN WELFARE INCOMP']
c_data.shape

(442415, 10)

In [ ]:
gdf_c_data = gpd.GeoDataFrame(c_data,geometry=gpd.points_from_xy(c_data.lon,c_data.lat),crs={'init':'EPSG:4326'})

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
t=taxi_zone.to_crs(epsg=4326)

In [ ]:
gdf_c_data

,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year,geometry
0,12/31/2019,17:30:00,32,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,40.820927,-73.943324,2019,POINT (-73.94332 40.82093)
2,12/15/2019,18:45:00,109,578,HARRASSMENT 2,VIOLATION,QUEENS,40.742281,-73.819824,2019,POINT (-73.81982 40.74228)
3,12/28/2019,01:00:00,47,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,40.875311,-73.847545,2019,POINT (-73.84755 40.87531)
4,12/27/2019,22:00:00,9,107,BURGLARY,FELONY,MANHATTAN,40.720759,-73.980466,2019,POINT (-73.98047 40.72076)
5,12/27/2019,20:10:00,79,117,DANGEROUS DRUGS,FELONY,BROOKLYN,40.695958,-73.937632,2019,POINT (-73.93763 40.69596)
...,...,...,...,...,...,...,...,...,...,...,...
446283,09/09/2019,08:30:00,18,578,HARRASSMENT 2,VIOLATION,MANHATTAN,40.767849,-73.987630,2019,POINT (-73.98763 40.76785)
446284,12/17/2019,10:00:00,113,109,GRAND LARCENY,FELONY,QUEENS,40.674593,-73.796869,2019,POINT (-73.79687 40.67459)
446285,08/21/2019,00:00:00,34,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,40.850460,-73.938470,2019,POINT (-73.93847 40.85046)
446286,12/31/2019,02:00:00,60,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,40.573909,-74.000495,2019,POINT (-74.00049 40.57391)


In [ ]:
#gdf=gdf_c_data.set_crs(epsg=4326,allow_override=True)
gdf_c_data_=gdf_c_data.sjoin(taxi_zone,how='left')


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year,geometry,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
0,12/31/2019,17:30:00,32,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,40.820927,-73.943324,2019,POINT (-73.94332 40.82093),43.0,0.000263896952605,42,0.0927092274616,42,Central Harlem North,Manhattan
2,12/15/2019,18:45:00,109,578,HARRASSMENT 2,VIOLATION,QUEENS,40.742281,-73.819824,2019,POINT (-73.81982 40.74228),191.0,0.000373808019026,192,0.11196461971,192,Queensboro Hill,Queens
3,12/28/2019,01:00:00,47,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,40.875311,-73.847545,2019,POINT (-73.84755 40.87531),82.0,0.00039956655019,81,0.197592771731,81,Eastchester,Bronx
4,12/27/2019,22:00:00,9,107,BURGLARY,FELONY,MANHATTAN,40.720759,-73.980466,2019,POINT (-73.98047 40.72076),3.0,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan
5,12/27/2019,20:10:00,79,117,DANGEROUS DRUGS,FELONY,BROOKLYN,40.695958,-73.937632,2019,POINT (-73.93763 40.69596),219.0,0.000310835395162,225,0.0887992073281,225,Stuyvesant Heights,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446283,09/09/2019,08:30:00,18,578,HARRASSMENT 2,VIOLATION,MANHATTAN,40.767849,-73.987630,2019,POINT (-73.98763 40.76785),47.0,0.0000942538425377,48,0.0437467441431,48,Clinton East,Manhattan
446284,12/17/2019,10:00:00,113,109,GRAND LARCENY,FELONY,QUEENS,40.674593,-73.796869,2019,POINT (-73.79687 40.67459),10.0,0.000435823818081,10,0.0998394794152,10,Baisley Park,Queens
446285,08/21/2019,00:00:00,34,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,40.850460,-73.938470,2019,POINT (-73.93847 40.85046),246.0,0.000359702747951,244,0.0805686044599,244,Washington Heights South,Manhattan
446286,12/31/2019,02:00:00,60,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,40.573909,-74.000495,2019,POINT (-74.00049 40.57391),54.0,0.000736274713586,55,0.149956807524,55,Coney Island,Brooklyn


In [ ]:
gdf_c_data_['BORO']=gdf_c_data_['borough'].apply(lambda x : str(x).upper())
gdf_c_data_['test']=gdf_c_data_['boro']==gdf_c_data_['BORO']

,Date,Time,loc ID,off type,off name,off law type,boro,lat,lon,year,geometry,index_right,shape_area,objectid,shape_leng,location_id,zone,borough,BORO,test
0,12/31/2019,17:30:00,32,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,40.820927,-73.943324,2019,POINT (-73.94332 40.82093),43.0,0.000263896952605,42,0.0927092274616,42,Central Harlem North,Manhattan,MANHATTAN,True
2,12/15/2019,18:45:00,109,578,HARRASSMENT 2,VIOLATION,QUEENS,40.742281,-73.819824,2019,POINT (-73.81982 40.74228),191.0,0.000373808019026,192,0.11196461971,192,Queensboro Hill,Queens,QUEENS,True
3,12/28/2019,01:00:00,47,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,40.875311,-73.847545,2019,POINT (-73.84755 40.87531),82.0,0.00039956655019,81,0.197592771731,81,Eastchester,Bronx,BRONX,True
4,12/27/2019,22:00:00,9,107,BURGLARY,FELONY,MANHATTAN,40.720759,-73.980466,2019,POINT (-73.98047 40.72076),3.0,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,MANHATTAN,True
5,12/27/2019,20:10:00,79,117,DANGEROUS DRUGS,FELONY,BROOKLYN,40.695958,-73.937632,2019,POINT (-73.93763 40.69596),219.0,0.000310835395162,225,0.0887992073281,225,Stuyvesant Heights,Brooklyn,BROOKLYN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446283,09/09/2019,08:30:00,18,578,HARRASSMENT 2,VIOLATION,MANHATTAN,40.767849,-73.987630,2019,POINT (-73.98763 40.76785),47.0,0.0000942538425377,48,0.0437467441431,48,Clinton East,Manhattan,MANHATTAN,True
446284,12/17/2019,10:00:00,113,109,GRAND LARCENY,FELONY,QUEENS,40.674593,-73.796869,2019,POINT (-73.79687 40.67459),10.0,0.000435823818081,10,0.0998394794152,10,Baisley Park,Queens,QUEENS,True
446285,08/21/2019,00:00:00,34,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,40.850460,-73.938470,2019,POINT (-73.93847 40.85046),246.0,0.000359702747951,244,0.0805686044599,244,Washington Heights South,Manhattan,MANHATTAN,True
446286,12/31/2019,02:00:00,60,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,40.573909,-74.000495,2019,POINT (-74.00049 40.57391),54.0,0.000736274713586,55,0.149956807524,55,Coney Island,Brooklyn,BROOKLYN,True


In [ ]:
gdf_c_data_=gdf_c_data_[gdf_c_data_['test']==True]
#this shows that there are more than 1800 records located in the edge of a borough
# we can simple drop them
gdf_c_data_.shape

(440612, 20)

In [ ]:
columns=['Date','Time','off type','off name','off law type','boro','year','geometry','location_id','zone']
gdf_c_data_=gdf_c_data_[columns]

In [ ]:
gdf_c_data_.shape

(440612, 10)

In [ ]:
gdf_c_data_.to_csv('gdf_c_data_2019_2020.csv')

In [ ]:
c_data.to_csv('c_data_2019_2020.csv')

In [ ]:
class_in_detail=pd.read_excel('/content/drive/MyDrive/ADS Crime and Uber/crime_classes_in_detail.xlsx')
class_in_detail.head()

,type,class,name
0,118.0,1.0,dangerous felony
1,101.0,1.0,dangerous felony
2,117.0,1.0,dangerous felony
3,106.0,1.0,dangerous felony
4,114.0,1.0,dangerous felony


In [ ]:
class_in_detail=pd.read_excel('/content/drive/MyDrive/ADS Crime and Uber/crime_classes_in_detail.xlsx')
class_in_detail.head()

In [ ]:
class_in_detail=pd.read_excel('crime_classes_in_detail.xlsx')
class_in_detail.head()

,type,class,name
0,118.0,1.0,dangerous felony
1,101.0,1.0,dangerous felony
2,117.0,1.0,dangerous felony
3,106.0,1.0,dangerous felony
4,114.0,1.0,dangerous felony


In [ ]:
class_in_detail['off type']=class_in_detail['type'].apply(lambda x : str(int(x)))
class_in_detail

,type,class,name,off type
0,118.0,1.0,dangerous felony,118
1,101.0,1.0,dangerous felony,101
2,117.0,1.0,dangerous felony,117
3,106.0,1.0,dangerous felony,106
4,114.0,1.0,dangerous felony,114
5,235.0,1.0,dangerous felony,235
6,236.0,1.0,dangerous felony,236
7,125.0,1.0,dangerous felony,125
8,124.0,1.0,dangerous felony,124
9,363.0,1.0,dangerous felony,363


In [ ]:
gdf_c_data_detail=gdf_c_data_.merge(class_in_detail,on='off type',how='left')

In [ ]:

gdf_c_data_detail

,Date,Time,off type,off name,off law type,boro,year,geometry,location_id,zone,type,class,name
0,12/31/2019,17:30:00,118,DANGEROUS WEAPONS,FELONY,MANHATTAN,2019,POINT (-73.94332 40.82093),42,Central Harlem North,118.0,1.0,dangerous felony
1,12/15/2019,18:45:00,578,HARRASSMENT 2,VIOLATION,QUEENS,2019,POINT (-73.81982 40.74228),192,Queensboro Hill,578.0,3.0,sex related
2,12/28/2019,01:00:00,126,MISCELLANEOUS PENAL LAW,FELONY,BRONX,2019,POINT (-73.84755 40.87531),81,Eastchester,126.0,6.0,others
3,12/27/2019,22:00:00,107,BURGLARY,FELONY,MANHATTAN,2019,POINT (-73.98047 40.72076),4,Alphabet City,107.0,2.0,Property related
4,12/27/2019,20:10:00,117,DANGEROUS DRUGS,FELONY,BROOKLYN,2019,POINT (-73.93763 40.69596),225,Stuyvesant Heights,117.0,1.0,dangerous felony
...,...,...,...,...,...,...,...,...,...,...,...,...,...
442875,09/09/2019,08:30:00,578,HARRASSMENT 2,VIOLATION,MANHATTAN,2019,POINT (-73.98763 40.76785),48,Clinton East,578.0,3.0,sex related
442876,12/17/2019,10:00:00,109,GRAND LARCENY,FELONY,QUEENS,2019,POINT (-73.79687 40.67459),10,Baisley Park,109.0,2.0,Property related
442877,08/21/2019,00:00:00,361,OFF. AGNST PUB ORD SENSBLTY &,MISDEMEANOR,MANHATTAN,2019,POINT (-73.93847 40.85046),244,Washington Heights South,361.0,6.0,others
442878,12/31/2019,02:00:00,121,CRIMINAL MISCHIEF & RELATED OF,FELONY,BROOKLYN,2019,POINT (-74.00049 40.57391),55,Coney Island,121.0,4.0,Offensive behavior


In [ ]:
gdf_drop_detail = gdf_c_data_detail.drop_duplicates()

In [ ]:
from datetime import date
def extract_year_month(x):
  g=x.split('/')
  year=int(g[2])
  month=int(g[0])
  day=int(g[1])

  return datetime(year,month,day)

In [ ]:
gdf_drop_detail['exact date']=gdf_drop_detail['Date'].apply(lambda x : extract_year_month(x))

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
gdf_drop_detail['month']=gdf_drop_detail['exact date'].dt.month

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
agg_month=gdf_drop_detail[['Time','year','location_id','name','month']].groupby(['year','location_id','name','month']).count().reset_index()

In [ ]:
agg_month.to_csv('agg_month_usable.csv')

In [ ]:
dangerous_list=agg_month[agg_month['Time']>150]['location_id']
taxi_zone[taxi_zone['location_id'].isin(dangerous_list)]

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
6,0.000389787989274,7,0.107417171123,7,Astoria,Queens,"MULTIPOLYGON (((-73.90414 40.76752, -73.90325 ..."
21,0.000322957654799,17,0.093522632948,17,Bedford,Brooklyn,"MULTIPOLYGON (((-73.94306 40.70142, -73.94220 ..."
34,0.000323825195932,35,0.085787918592,35,Brownsville,Brooklyn,"MULTIPOLYGON (((-73.90105 40.66212, -73.90066 ..."
36,0.000452062644782,37,0.142810040466,37,Bushwick South,Brooklyn,"MULTIPOLYGON (((-73.93313 40.70470, -73.93301 ..."
40,0.000143093037737,41,0.052793109453,41,Central Harlem,Manhattan,"MULTIPOLYGON (((-73.94774 40.80960, -73.94506 ..."
43,0.000263896952605,42,0.0927092274616,42,Central Harlem North,Manhattan,"MULTIPOLYGON (((-73.93436 40.83620, -73.93445 ..."
54,0.000736274713586,55,0.149956807524,55,Coney Island,Brooklyn,"MULTIPOLYGON (((-73.98672 40.58457, -73.98628 ..."
60,0.000510772092295,61,0.117310808097,61,Crown Heights North,Brooklyn,"MULTIPOLYGON (((-73.92916 40.68344, -73.92901 ..."
75,0.000295038418204,74,0.11029093625,74,East Harlem North,Manhattan,"MULTIPOLYGON (((-73.93381 40.81651, -73.93383 ..."
79,0.00126660143241,76,0.241203016269,76,East New York,Brooklyn,"MULTIPOLYGON (((-73.88064 40.67918, -73.88061 ..."


In [ ]:
trips_1907 = pd.read_csv('yellow_tripdata_2019-07.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
trips_1907.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2019-07-01 00:51:04,2019-07-01 00:51:33,1.0,0.00,1.0,N,193,193,1.0,2.5,0.5,0.5,1.14,0.00,0.3,4.94,0.0
1,2.0,2019-07-01 00:46:04,2019-07-01 01:05:46,1.0,4.16,1.0,N,234,25,2.0,16.5,0.5,0.5,0.00,0.00,0.3,20.30,2.5
2,1.0,2019-07-01 00:25:09,2019-07-01 01:00:56,1.0,18.80,2.0,N,132,42,1.0,52.0,0.0,0.5,11.75,6.12,0.3,70.67,0.0
3,2.0,2019-07-01 00:33:32,2019-07-01 01:15:27,1.0,18.46,2.0,N,132,142,1.0,52.0,0.0,0.5,11.06,0.00,0.3,66.36,2.5
4,1.0,2019-07-01 00:00:55,2019-07-01 00:13:05,0.0,1.70,1.0,N,107,114,1.0,9.5,3.0,0.5,2.00,0.00,0.3,15.30,2.5


In [ ]:
trips_1907_group=trips_1907[['PULocationID','tpep_dropoff_datetime']].groupby('PULocationID').count().reset_index()

In [ ]:
trips_1907_group


,PULocationID,tpep_dropoff_datetime
0,1,703
1,2,19
2,3,200
3,4,9637
4,5,49
...,...,...
256,261,40635
257,262,67591
258,263,107630
259,264,49261


In [ ]:
trips=trips_1907
for i in ['08','09','10','11','12']:
  trips_monthly = pd.read_csv('yellow_tripdata_2019-{}.csv'.format(i))

  trips=pd.concat([trips,trips_monthly])


trips.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
